In [ ]:
!pip install tweepy


In [2]:
!pip install demoji


In [3]:
pip install demoji


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [4]:
"""
preprocess-twitter.py

python preprocess-twitter.py "Some random text with #hashtags, @mentions and http://t.co/kdjfkdjf (links). :)"

Script for preprocessing tweets by Romain Paulus
with small modifications by Jeffrey Pennington
with translation to Python by Motoki Wu

Translation of Ruby script to create features for GloVe vectors for Twitter data.
http://nlp.stanford.edu/projects/glove/preprocess-twitter.rb
"""

import regex as re
import tweepy

FLAGS = re.MULTILINE | re.DOTALL


def aggregateUpper(hashtag_list):
    hashtag_list_copy = hashtag_list
    length = [len(i) for i in hashtag_list_copy]
    if length[0]==0:
        hashtag_list_copy.pop(0)
        length.pop(0) 
    change = True
    try:
        while change:
            pos = length.index(1)
            if length[pos+1]==1:
                hashtag_list_copy[pos] = hashtag_list_copy[pos]+hashtag_list_copy[pos+1]
                hashtag_list_copy.pop(pos+1)
                length.pop(pos+1)
                change = True
            else:
                change = False
    except:
        pass
    return hashtag_list_copy

def hashtag(text):
    text = text.group()
    hashtag_body = text[1:]
    if hashtag_body.isupper():
        result = " {} ".format(hashtag_body.lower())
    else:
        result = " ".join(["<hashtag>"] + aggregateUpper(re.split(r"(?=[A-Z])", hashtag_body, flags=FLAGS)))  # add aggregateUpper Function
    return result
    
def allcaps(text):
    text = text.group()
    return text.lower() + " <allcaps>"

def tokenize(text):
    # Different regex parts for smiley faces
    eyes = r"[8:=;]"
    nose = r"['`\-]?"

    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)

    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<url>")
    text = re_sub(r"@\w+", '<user>')
    # text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), "<smile>")
    # text = re_sub(r"{}{}p+".format(eyes, nose), "<lolface>")
    # text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), "<sadface>")
    # text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), "<neutralface>")
    text = re_sub(r"/"," / ")
    # text = re_sub(r"<3","<heart>")
   # text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<number>") # defunction number
    text = re_sub(r"#\S+", hashtag)
    text = re_sub(r"([!?.]){2,}", r"\1 <repeat>")
    text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong>")

    ## -- I just don't understand why the Ruby script adds <allcaps> to everything so I limited the selection.
    # text = re_sub(r"([^a-z0-9()<>'`\-]){2,}", allcaps)
    # text = re_sub(r"([A-Z]){2,}", allcaps)   # defunction <allcap>

    return text.lower()

def get_keys():
    keys = {}
    with open("apikeys.txt") as f:
        api_key = f.read().split("\n")
    keys['consumer key'] = api_key[0]
    keys['consumer secret'] = api_key[1]
    keys['access token'] = api_key[2]
    keys['access token secret'] = api_key[3]
    return keys



def domain(url):
    pattern = r"https?://([\w.-]+)/?"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    else:
        return

def initialize_api():
    keys = get_keys()
    auth = tweepy.OAuthHandler(keys['consumer key'], keys['consumer secret'])
    auth.set_access_token(keys['access token'], keys['access token secret'])
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    return api



In [5]:
import pymongo
import regex as re
from datetime import datetime, date, timedelta
import demoji
import emoji
demoji.download_codes()

import tweepy
from platform import python_version
print("Python version:", python_version())

... OK (Got response in 0.23 seconds)
Writing emoji data to /Users/hawhee/.demoji/codes.json ...
... OK
Python version: 3.9.2


In [6]:
def saveToMongo(collection, document):
    col = db[collection]
    col.insert_one(document)

def clearEntireCollection(collection):
    col = db[collection]
    col.delete_many({})
    
def domain(url):
    pattern = r"https?://([\w.-]+)/?"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    else:
        return
    
    
def tranform_tweets(tweet, collection):
    hashtags, mentions, urls, urlDomains, emojis = [], [], [], [], []
            
    time = tweet.created_at
    tweetID = tweet.id_str
    userID = tweet.user.id_str
    screenName = tweet.user.screen_name
    text = tweet.full_text

    # preprocessing
    emojis.extend(demoji.findall(text).keys())
    text2 = demoji.replace(text,'')
    textPrep = tokenize(text2)

    try:
        isoLanguage = tweet.metadata["iso_language_code"]
    except:
        isoLanguage = None
    favoriteCount = tweet.favorite_count
    retweetCount = tweet.retweet_count
#   replyCount = tweet.reply_count                      # Premium API Only
    location = tweet.user.location                        # Location on the user profile
    if tweet.place:
        placeName = tweet.place.full_name                     # Place of the post that is tagged with
        placeCountry = tweet.place.country_code 
    else:
        placeName = None
        placeCountry = None
    followersCount = tweet.user.followers_count

    # unwraping
    hashtags_tweepy = tweet.entities['hashtags'] 
    mentions_tweepy = tweet.entities['user_mentions']
    urls_tweepy = tweet.entities['urls']
    for hashtag in hashtags_tweepy: 
        hashtags.append(hashtag['text'].lower()) 
    for user in mentions_tweepy: 
        mentions.append(user['screen_name']) 
    for url in urls_tweepy: 
        urls.append(url['expanded_url'].lower())
        urlDomains.append(domain(url['expanded_url'].lower()))

    document = {
        "time": time,
        "tweetID": tweetID,
        "userID": userID,
        "screenName": screenName,
        "text": text,
        "emojis": emojis,
        "textPrep": textPrep,
        "isoLanguage": isoLanguage,
        "favoriteCount": favoriteCount,
        "retweetCount": retweetCount,
        "location": location,
        "placeName": placeName,
        "placeCountry": placeCountry,
        "followersCount": followersCount,
        "hashtags": hashtags,
        "mentions": mentions,
        "urls": urls,
        "urlDomains": urlDomains
    }

    saveToMongo(collection, document)
    return userID


def tranform_users(tweet, user_collection):
    userID = tweet.user.id_str
    name = tweet.user.name
    screenName = tweet.user.screen_name
    location = tweet.user.location
    description = tweet.user.description
    following = tweet.user.friends_count
    follower = tweet.user.followers_count
    favorite = tweet.user.favourites_count
    status = tweet.user.statuses_count
    profile_image_url = tweet.user.profile_image_url

    document2 = {
            "userID": userID,
            "name": name,
            "screenName": screenName,
            "location": location,
            "description": description,
            "following": following,
            "follower": follower,
            "favorite": favorite,
            "status": status,
            "profile_image_url": profile_image_url
        }
    saveToMongo(user_collection, document2)
    

    
    
    
    
def search_tweets(api, collection, query, since, until, user_collection=None, place_id=None):
    if place_id:
        q = query + f" since:{since} until:{until}" + " -filter:retweets AND -filter:replies" + f" AND place:{place_id}"
        print(f"Start Collecting Query: {q}")
    else:
        q = query + f" since:{since} until:{until}" + " -filter:retweets AND -filter:replies"
        print(f"Start Collecting Query: {q}")
    
    
    count_ = 0
    print(f"Time of now: {datetime.now()}")
    
    
    for tweet in tweepy.Cursor(api.search, q=q, lang = "en", tweet_mode="extended").items(30):
        count_ += 1
        if count_%100 == 0:
            print(f"Have collecte {count_} tweets! - {datetime.now()}")

        # check if this tweet is already in the database
        if db[collection].find_one({"tweetID": tweet.id_str}):
            continue
        else:
            userID = tranform_tweets(tweet, collection)
            
            # check if this user is already in the database
            if db[user_collection].find_one({"userID": userID}):
                continue
            else:
                tranform_users(tweet, user_collection)
                # Users' tweets: Only scrape at most 100 tweets or the tweets within 6 months (180 days)
                for tweet in tweepy.Cursor(api.user_timeline,id=userID, tweet_mode='extended').items(100):
                    created = tweet.created_at
                    if (created < datetime.now() - timedelta(days=180)) or (db[collection].find_one({"tweetID": tweet.id_str})):
                        break
                    else:
                        tranform_tweets(tweet, collection)

In [9]:
keys = get_keys()

In [10]:
### Connect to mongoDB
client = pymongo.MongoClient(mongoDB_connection)
db = client[database_name]
db.list_collection_names()

['taccounts', 'users', 'tweets']

In [11]:
### Connect to Twitter API
api = initialize_api()


In [26]:
### Search filters
collection = "tweets"
user_collection = "taccounts"
query = "BLM"
since = str(date.today()-timedelta(days=3))
until = str(date.today()-timedelta(days=2))
searchCountry = "United States"

places = api.geo_search(query=searchCountry, granularity="country")
for place in places:
    if place.full_name == searchCountry:
        place_obj = place
        place_id = place.id
        
print(f"Collect data with keywords {query} from {since} to {until} in {place_obj.name}({place_id})")
print(f"Data collected will be put into collection {collection}")

Collect data with keywords BLM from 2021-06-19 to 2021-06-20 in United States(96683cc9126741d1)
Data collected will be put into collection tweets


In [ ]:
### Collecting
# search_tweets(api, collection, query, since, until, place_id # with specifc location
search_tweets(api, collection, query, since, until, user_collection=user_collection)

In [ ]:
print("Documents:",db[collection].count_documents({}))
print("Documents:",db[user_collection].count_documents({}))

In [21]:
db[collection].find_one({"time":{"$lt":datetime(2021,5,30,0,0,0)}})
# db[collection].find_one({"favoriteCount":{"$gt":1}})

In [ ]:
### Cleaning
clearEntireCollection("tweets")
print(db.tweets.find_one()) #check if it is empty